# 목차

0. Preprocessing
1. Bagging
    - 1.a 직접 구현
        - [참고논문: Bagging Predictors](https://www.stat.berkeley.edu/~breiman/bagging.pdf)
    - 1.b. 라이브러리 활용
2. Random Forest
    - 2.a 직접 구현
        - [참고논문: Random Forest](https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf)
    - 2.b. 라이브러리 활용
3. Boosting
    - 3.1 XGBoost
        - 3.1.a 직접 구현
            - [참고논문: XGBoost: A Scalable Tree Boosting System](https://arxiv.org/pdf/1603.02754.pdf)
        - 3.1.b 라이브러리 활용
    - 3.2 LightGBM
        - 3.2.a 직접 구현
            - [참고논문: LightGBM: A Highly Efficient Gradient Boosting Decision Tree](https://papers.nips.cc/paper/2017/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf)
        - 2.2.b 라이브러리 활용
    - 3.3 CatBoost
<!--         - 3.3.a 직접 구현
            - [참고논문: CatBoost: Unbiased Boosting with Categorical Features](https://arxiv.org/pdf/1706.09516.pdf) -->
        - 3.3.b 라이브러리 활용
4. 결과 및 느낀점

- *DSBA 연구실 강의와 투빅스 자료를 함께 참고하였습니다.*

---

# 0. Preprocessing

In [1]:
# 데이터 적재 및 전처리를 위한 라이브러리
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score

# 직접 구현 관련 라이브러리
import itertools
from multiprocessing import Pool, cpu_count
from sklearn.model_selection import KFold

# 모델을 포함한 라이브러리
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [2]:
# 데이터 로드
file_path = "./BankChurners.csv"
data = pd.read_csv(file_path)

# 처음 5개의 행을 출력하여 데이터를 확인
data.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [3]:
# 불필요한 컬럼 삭제
data = data.drop(data.columns[-2:], axis=1)

# 목표 변수 인코딩: 'Existing Customer'를 0, 'Attrited Customer'를 1로 인코딩
target_encoder = LabelEncoder()
data['Attrition_Flag'] = target_encoder.fit_transform(data['Attrition_Flag'])

# 범주형 피처 인코딩
categorical_features = data.select_dtypes(include=['object']).columns
for feature in categorical_features:
    encoder = LabelEncoder()
    data[feature] = encoder.fit_transform(data[feature])

# 데이터 분할: 학습 및 테스트 데이터셋으로 분할
X = data.drop('Attrition_Flag', axis=1)  # 피처
y = data['Attrition_Flag']               # 목표 변수
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 1. Bagging

## 1.a. 직접구현

#### 특징:
1. Bagging 방법을 사용하여 여러 개의 결정 트리를 학습합니다.
2. 각 트리에 대해 샘플과 특징을 랜덤하게 선택하여 학습시킵니다.

#### 주요 변수:
- `n_estimators`: 학습할 결정 트리의 수
- `max_samples`: 각 트리를 학습할 때 사용할 샘플의 비율
- `max_features`: 각 트리를 학습할 때 사용할 특징의 비율
- `trees`: 학습된 결정 트리와 해당 트리를 학습할 때 사용된 특징의 인덱스를 저장하는 리스트

#### 주요 메서드:

1. `fit(X, y)`:
    - 입력 데이터 `X`와 레이블 `y`를 사용하여 여러 개의 결정 트리를 학습시킵니다.
    - 각 트리를 학습할 때는 `max_samples` 비율의 샘플과 `max_features` 비율의 특징을 랜덤하게 선택하여 사용합니다.
    - 학습된 결정 트리와 사용된 특징의 인덱스는 `trees` 리스트에 저장됩니다.

2. `predict(X)`:
    - 입력 데이터 `X`에 대한 예측을 수행합니다.
    - 각 결정 트리에서 예측을 수행하고, 최종 예측은 모든 트리의 예측 결과를 다수결 방식으로 결정합니다.

#### 동작 방식:
1. Bagging 방법을 사용하여 여러 개의 결정 트리를 학습합니다. 각 트리는 데이터의 일부 샘플과 특징을 랜덤하게 선택하여 학습되기 때문에, 각 트리는 서로 다른 특성을 가집니다.
2. 예측 시, 모든 트리의 예측 결과를 모아 다수결 방식으로 최종 예측을 결정합니다. 이렇게 하면 개별 트리의 예측 오차를 줄일 수 있습니다.

In [4]:
class BaggingImplementation:
    def __init__(self, n_estimators=20, max_samples=0.8, max_features=0.8):
        self.n_estimators = n_estimators  # base learner의 개수
        self.max_samples = max_samples  # 각 base learner에 대한 샘플링 비율
        self.max_features = max_features  # 각 base learner에 대한 특징 선택 비율
        self.trees = []  # base learner들을 저장할 리스트

    def fit(self, X, y):
        X = np.array(X)  # X를 Numpy array로 변환
        y = np.array(y)  # y를 Numpy array로 변환
        
        n_samples, n_features = X.shape  # 데이터의 샘플 수와 특징 수를 구함
        
        for _ in range(self.n_estimators):  # 지정된 개수만큼 base learner를 학습
            # 샘플과 특징을 랜덤하게 선택
            sample_idxs = np.random.choice(n_samples, int(n_samples * self.max_samples), replace=True)
            feature_idxs = np.random.choice(n_features, int(n_features * self.max_features), replace=False)
            
            # 선택된 샘플과 특징으로 데이터를 구성
            X_subset = X[sample_idxs][:, feature_idxs]
            y_subset = y[sample_idxs]
            
            # 결정 트리를 학습 (다른 알고리즘으로도 대체 가능)
            # 단, 복잡도가 높은 모델이 높은 성능 기대
            tree = DecisionTreeClassifier()
            tree.fit(X_subset, y_subset)
            
            # 학습된 tree와 사용된 특징의 인덱스를 저장
            self.trees.append((tree, feature_idxs))
            
    def predict(self, X):
        # 각 base learner의 예측을 수집
        X = np.array(X)  # X를 Numpy array로 변환
        predictions = []
        for tree, feature_idxs in self.trees:
            X_subset = X[:, feature_idxs]  # 사용된 특징만 선택
            predictions.append(tree.predict(X_subset))  # 예측 수행
            
        # 수집된 예측을 통해 최종 예측을 결정 (다수결 방식)
        predictions = np.array(predictions)
        final_predictions = []
        for i in range(X.shape[0]):
            final_predictions.append(np.bincount(predictions[:, i]).argmax())
        
        return np.array(final_predictions)

In [5]:
# 학습
bagging_implemantation = BaggingImplementation()

start_time_train = time.time()
bagging_implemantation.fit(X_train.values, y_train.values)
end_time_train = time.time()
bagging_implemantation_training_time = end_time_train - start_time_train

# 테스트 데이터에 대한 예측 수행
bagging_implemantation_predictions = bagging_implemantation.predict(X_test)

# 정확도 계산
bagging_implemantation_accuracy = accuracy_score(y_test.values, bagging_implemantation_predictions)

In [6]:
# 분류 정확도 계산 및 시간
print('bagging_implemantation_accuracy:',bagging_implemantation_accuracy)
print('bagging_implemantation_training_time:',bagging_implemantation_training_time)

bagging_implemantation_accuracy: 0.9610069101678184
bagging_implemantation_training_time: 0.5716240406036377


## 1.b. 라이브러리 활용

In [7]:
# Base estimator로 결정 트리를 사용
base_estimator = DecisionTreeClassifier()

# Bagging 분류기 생성
bagging_clf = BaggingClassifier(base_estimator=base_estimator, n_estimators=50, random_state=42)


# 모델 학습
start_time_train_bagging = time.time()  # 학습 시작 시간 저장
bagging_clf.fit(X_train, y_train)
end_time_train_bagging = time.time()  # 학습 종료 시간 저장
scikitlearn_bagging_training_time= end_time_train_bagging - start_time_train_bagging

# 테스트 데이터에 대한 예측 수행
scikitlearn_bagging_predictions =bagging_clf.predict(X_test)

# 분류 정확도 계산
scikitlearn_bagging_accuracy = accuracy_score(y_test, scikitlearn_bagging_predictions)

In [8]:
# 분류 정확도 계산 및 시간
print('scikitlearn_bagging_accuracy:',scikitlearn_bagging_accuracy)
print('scikitlearn_bagging_training_time:',scikitlearn_bagging_training_time)

scikitlearn_bagging_accuracy: 0.9580454096742349
scikitlearn_bagging_training_time: 1.455275058746338


# 2. Random Forest

## 2.a. 직접 구현

#### 특징:
1. 여러 개의 결정 트리를 학습하여 앙상블 방식으로 분류 작업을 수행합니다.
2. 각 트리는 부트스트랩 샘플링 방법으로 생성된 서브셋에서 학습됩니다.
3. 예측 시, 모든 트리의 예측 결과를 모아 다수결 방식으로 최종 예측을 결정합니다.

#### 주요 변수:
- `n_estimators`: 학습할 결정 트리의 수
- `max_depth`: 각 트리의 최대 깊이
- `max_features`: 각 트리를 학습할 때 선택될 특징의 최대 수 (예: "sqrt"는 전체 특징 수의 제곱근만큼의 특징을 선택)
- `min_samples_split`: 노드를 분할하기 위한 최소한의 샘플 수
- `trees`: 학습된 결정 트리들을 저장하는 리스트

#### 메서드:

1. `fit(X, y)`:
    - 입력 데이터 `X`와 레이블 `y`를 사용하여 여러 개의 결정 트리를 학습시킵니다.
    - 각 트리는 부트스트랩 샘플링 방법으로 생성된 데이터 서브셋에서 학습됩니다.
    - 학습된 결정 트리는 `trees` 리스트에 저장됩니다.

2. `predict(X)`:
    - 입력 데이터 `X`에 대한 예측을 수행합니다.
    - 각 결정 트리에서 예측을 수행하고, 최종 예측은 모든 트리의 예측 결과를 모아 다수결 방식으로 결정합니다.

#### 동작 방식:
1. 랜덤 포레스트는 앙상블 방식으로 분류 작업을 수행하는 데 여러 개의 결정 트리를 사용합니다. 각 트리는 데이터의 부트스트랩 샘플에서 학습되므로, 각 트리는 서로 다른 특성을 가집니다.
2. 예측 시, 모든 트리의 예측 결과를 모아서 다수결 방식으로 최종 예측을 결정합니다. 이렇게 하면 개별 트리의 예측 오차를 줄일 수 있으며, 전체적인 예측 성능이 향상됩니다.

In [9]:
class RandomForestImplementation:
    def __init__(self, n_estimators=20, max_depth=None, max_features='sqrt', min_samples_split=2):
        self.n_estimators = n_estimators  # 트리의 개수
        self.max_depth = max_depth  # 트리의 최대 깊이
        self.max_features = max_features  # 각 노드에서 선택할 특성의 개수
        self.min_samples_split = min_samples_split  # 노드를 분할하기 위한 최소 샘플 수
        self.trees = []  # 학습된 트리들을 저장할 리스트

    def fit(self, X, y):
        X = np.array(X)  # X를 Numpy array로 변환
        y = np.array(y)  # y를 Numpy array로 변환
        
        n_samples, n_features = X.shape  # 데이터의 샘플 수와 특징 수를 구함
        
        for _ in range(self.n_estimators):
            # 부트스트랩 샘플 생성
            sample_idxs = np.random.choice(n_samples, n_samples, replace=True)
            X_bootstrap = X[sample_idxs]
            y_bootstrap = y[sample_idxs]
            
            # 결정 트리 생성 및 학습
            tree = DecisionTreeClassifier(max_depth=self.max_depth, 
                                          max_features=self.max_features, 
                                          min_samples_split=self.min_samples_split)
            tree.fit(X_bootstrap, y_bootstrap)
            
            # 학습된 트리 저장
            self.trees.append(tree)
            
    def predict(self, X):
        X = np.array(X)  # X를 Numpy array로 변환
        predictions = np.zeros(len(X))
        
        # 각 트리의 예측을 수집
        for tree in self.trees:
            predictions += tree.predict(X)
        
        # 수집된 예측을 통해 최종 예측을 결정 (다수결 방식)
        final_predictions = (predictions / self.n_estimators >= 0.5).astype(int)
        
        return final_predictions

In [10]:
# 학습
RF_implemantation = RandomForestImplementation()

start_time_train = time.time()
RF_implemantation.fit(X_train.values, y_train.values)
end_time_train = time.time()
RF_implemantation_training_time = end_time_train - start_time_train

# 테스트 데이터에 대한 예측 수행
RF_implemantation_predictions = RF_implemantation.predict(X_test)

# 정확도 계산
RF_implemantation_accuracy = accuracy_score(y_test.values, RF_implemantation_predictions)

In [11]:
# 분류 정확도 계산 및 시간
print('RF_implemantation_accuracy:',RF_implemantation_accuracy)
print('RF_implemantation_training_time:',RF_implemantation_training_time)

RF_implemantation_accuracy: 0.9580454096742349
RF_implemantation_training_time: 0.2165968418121338


## 2.b. 라이브러리 활용

In [12]:
# 학습
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

start_time_train = time.time()
random_forest_clf.fit(X_train, y_train)
end_time_train = time.time()
training_time = end_time_train - start_time_train

# 테스트 데이터에 대한 예측 수행
y_pred = random_forest_clf.predict(X_test)

# 분류 정확도 계산
scikitlearn_RF_accuracy = accuracy_score(y_test, y_pred)

In [13]:
# 분류 정확도 계산 및 시간
print('scikitlearn_RF_accuracy:',scikitlearn_RF_accuracy)
print('scikitlearn_RF_training_time:',training_time)

scikitlearn_RF_accuracy: 0.9619940769990128
scikitlearn_RF_training_time: 0.7276842594146729


# 3. Boosting

## 3.1. XGBoost

### 3-1-a. 직접 구현하기

#### 특징:
1. 부스팅 방법을 사용하여 여러 개의 결정 트리를 연속적으로 학습합니다.
2. 각 트리는 이전 트리의 예측 오차를 바탕으로 학습되며, 이를 통해 예측 성능을 향상시킵니다.
3. L2 정규화를 사용하여 모델의 복잡도를 제한하고 과적합을 방지합니다.

#### 주요 변수:
- `n_estimators`: 학습할 결정 트리의 수
- `learning_rate`: 각 트리의 기여도를 조절하기 위한 학습률
- `max_depth`: 각 트리의 최대 깊이
- `min_size`: 리프 노드가 가져야 할 최소한의 샘플 수
- `subsample`: 각 트리를 학습할 때 사용할 샘플의 비율
- `colsample`: 각 트리를 학습할 때 사용할 특징의 비율
- `trees`: 학습된 결정 트리들을 저장하는 리스트
- `lambda_reg`: L2 정규화 항

#### 메서드:

1. `fit(X, y)`:
    - 입력 데이터 `X`와 레이블 `y`를 사용하여 여러 개의 결정 트리를 학습시킵니다.
    - 각 트리는 이전 트리의 예측 오차를 바탕으로 학습되며, `subsample`과 `colsample`을 통해 데이터의 일부 샘플과 특징을 랜덤하게 선택하여 사용합니다.
    - 학습된 결정 트리는 `trees` 리스트에 저장됩니다.

2. `predict(X)`:
    - 입력 데이터 `X`에 대한 예측을 수행합니다.
    - 각 결정 트리의 예측값을 모두 합산하여 최종 예측을 얻습니다.

3. `split_dataset(X, y, feature_index, threshold)`:
    - 주어진 특징과 임계값을 기준으로 데이터셋을 분할합니다.
    - 왼쪽 및 오른쪽 자식 노드의 데이터를 반환합니다.

4. `calculate_gini(y)`:
    - 주어진 레이블 데이터에 대한 지니 불순도를 계산합니다.

5. `gini_with_regularization(y, num_samples)`:
    - 주어진 레이블 데이터에 대한 지니 불순도를 계산하되, L2 정규화 항을 추가하여 반환합니다.

6. `get_best_split(X, y)`:
    - 주어진 데이터를 기반으로 최적의 분할 특징과 임계값을 찾아 반환합니다.

7. `build_decision_tree(X, y, current_depth=0)`:
    - 주어진 데이터를 기반으로 결정 트리를 재귀적으로 구성합니다.
    - 최대 깊이나 최소 샘플 크기에 도달하면 종료합니다.

8. `predict_with_tree(tree, x)`:
    - 주어진 결정 트리를 사용하여 입력 데이터 포인트에 대한 예측값을 계산합니다.

9. `cross_validate(X, y, k=5)`:
    - k-fold 교차 검증을 수행하여 모델의 정확도를 평가합니다.

10. `parallel_tree_building(data)`:
    - 멀티프로세싱을 사용하여 병렬로 트리를 구성합니다.

#### 동작 방식:
1. XGBoost는 부스팅 방법을 사용하여 여러 개의 결정 트리를 연속적으로 학습시킵니다. 각 트리는 이전 트리들의 예측 오차를 바탕으로 학습되기 때문에, 연속적인 학습을 통해 예측 성능이 향상됩니다.
2. 각 트리의 학습 시, 데이터의 일부 샘플과 특징을 랜덤하게 선택하여 학습되며, 이를 통해 모델의 일반화 성능이 향상됩니다.
3. L2 정규화를 사용하여 각 트리의 복잡도를 제한하고, 과적합을 방지합니다.

In [14]:
class XGBoost_Implemantation:
    def __init__(self, n_estimators=20, learning_rate=0.1, max_depth=3, min_size=5, subsample=0.8, colsample=0.8, lambda_reg=1):
        # 초기화 함수에서 모델의 하이퍼파라미터를 설정합니다.
        
        self.n_estimators = n_estimators  # 부스팅에서 사용될 트리의 개수
        self.learning_rate = learning_rate  # 각 트리의 기여도를 조절
        self.max_depth = max_depth  # 각 트리의 최대 깊이
        self.min_size = min_size  # 리프 노드가 가져야 할 최소한의 샘플 수
        self.subsample = subsample  # 샘플링 비율
        self.colsample = colsample  # 피처 샘플링 비율
        self.trees = []  # 학습된 트리들을 저장할 리스트
        self.lambda_reg = lambda_reg  # L2 정규화를 위한 값

    def fit(self, X, y):
        # 모델 학습 함수
        
        # 초기 예측값을 설정. 일반적으로 타겟의 평균값으로 시작
        self.prediction = np.full(y.shape, np.mean(y))
        
        # 병렬 처리를 위해 멀티프로세싱을 사용하여 각 트리를 독립적으로 구축
        pool = Pool(processes=cpu_count())
        residuals = [y - self.prediction for _ in range(self.n_estimators)]
        results = pool.map(self.parallel_tree_building, [(X, res) for res in residuals])
        pool.close()
        self.trees.extend(results)
        
        # 각 트리로부터 얻은 예측값을 기존의 예측값에 더함
        for tree in results:
            tree_predictions = np.array([self.predict_with_tree(tree, x) for x in X])
            self.prediction += self.learning_rate * tree_predictions

    def predict(self, X):
        # 학습된 모델로부터 예측값을 얻는 함수
        
        # 초기 예측값 설정
        final_predictions = np.full(X.shape[0], np.mean(self.prediction))
        
        # 각 트리로부터 얻은 예측값을 누적하여 최종 예측값을 구함
        for tree in self.trees:
            tree_predictions = np.array([self.predict_with_tree(tree, x) for x in X])
            final_predictions += self.learning_rate * tree_predictions
            
        # 분류 문제의 경우, 임계치를 기준으로 예측 클래스를 결정
        return (final_predictions > 0.5).astype(int)

    def split_dataset(self, X, y, feature_index, threshold):
        # 주어진 피처와 임계값을 기준으로 데이터셋을 분할하는 함수
        
        # 누락된 데이터 처리
        if np.isnan(threshold):
            left_mask = np.isnan(X[:, feature_index])
            right_mask = ~left_mask
        else:
            left_mask = X[:, feature_index] < threshold
            right_mask = ~left_mask
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    def calculate_gini(self, y):
        # 주어진 데이터의 지니 불순도를 계산하는 함수
        
        classes = np.unique(y)
        gini = 1.0
        for c in classes:
            p = np.sum(y == c) / len(y)
            gini -= p**2
        return gini

    def gini_with_regularization(self, y, num_samples):
        # 정규화 항을 포함한 지니 불순도를 계산하는 함수
        
        gini = self.calculate_gini(y)
        return gini + self.lambda_reg / num_samples

    def get_best_split(self, X, y):
        # 주어진 데이터에서 가장 좋은 분할을 찾는 함수
        
        best_gini = float('inf')
        best_feature_index = None
        best_threshold = None
        for feature_index in range(X.shape[1]):
            unique_values = np.unique(X[:, feature_index])
            for threshold in unique_values:
                X_left, y_left, X_right, y_right = self.split_dataset(X, y, feature_index, threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                left_gini = self.gini_with_regularization(y_left, len(X_left))
                right_gini = self.gini_with_regularization(y_right, len(X_right))
                gini = (len(y_left) * left_gini + len(y_right) * right_gini) / len(y)
                if gini < best_gini:
                    best_gini = gini
                    best_feature_index = feature_index
                    best_threshold = threshold
        return best_feature_index, best_threshold

    def build_decision_tree(self, X, y, current_depth=0):
        # 재귀적으로 결정 트리를 구성하는 함수
        
        if current_depth >= self.max_depth or len(y) <= self.min_size:
            return {'prediction': np.mean(y)}
        feature_index, threshold = self.get_best_split(X, y)
        if feature_index is None:
            return {'prediction': np.mean(y)}
        X_left, y_left, X_right, y_right = self.split_dataset(X, y, feature_index, threshold)
        return {
            'feature_index': feature_index,
            'threshold': threshold,
            'left': self.build_decision_tree(X_left, y_left, current_depth + 1),
            'right': self.build_decision_tree(X_right, y_right, current_depth + 1)
        }

    def predict_with_tree(self, tree, x):
        # 결정 트리를 사용하여 개별 데이터 포인트의 예측값을 계산하는 함수
        
        if 'prediction' in tree:
            return tree['prediction']
        feature_index = tree['feature_index']
        threshold = tree['threshold']
        if (np.isnan(threshold) and np.isnan(x[feature_index])) or (x[feature_index] < threshold):
            return self.predict_with_tree(tree['left'], x)
        else:
            return self.predict_with_tree(tree['right'], x)

    def cross_validate(self, X, y, k=5):
        # k-fold 교차 검증을 수행하는 함수
        
        kf = KFold(n_splits=k)
        scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            self.fit(X_train, y_train)
            y_pred = self.predict(X_test)
            accuracy = np.mean(y_pred == y_test)
            scores.append(accuracy)
        return scores

    def parallel_tree_building(self, data):
        # 병렬 처리를 위한 트리 구성 함수
        
        X, residuals = data
        return self.build_decision_tree(X, residuals, current_depth=0)

In [15]:
# 학습
xgb_implemantation = XGBoost_Implemantation(n_estimators=5, learning_rate=0.1, max_depth=3, min_size=5, subsample=0.8, colsample=0.8)

start_time_train = time.time()
xgb_implemantation.fit(X_train.values, y_train.values)
end_time_train = time.time()
xgb_implemantation_training_time = end_time_train - start_time_train

# 테스트 데이터에 대한 예측 수행
xgb_implemantation_predictions = xgb_implemantation.predict(X_test.values)

# 정확도 계산
xgb_implemantation_accuracy = accuracy_score(y_test.values, xgb_implemantation_predictions)

In [16]:
# 분류 정확도 계산 및 시간
print('xgb_implemantation_accuracy:',xgb_implemantation_accuracy)
print('xgb_implemantation_training_time:',xgb_implemantation_training_time)

### 3-1-b. 라이브러리 활용

In [17]:
# 학습
scikitlearn_xgb_model = XGBClassifier(n_estimators=5, learning_rate=0.1, max_depth=3)

start_time_train_xgb = time.time()  # 학습 시작 시간 저장
scikitlearn_xgb_model.fit(X_train.values, y_train.values)  # 모델 학습
end_time_train_xgb = time.time()  # 학습 종료 시간 저장
scikitlearn_xgb_training_time= end_time_train_xgb - start_time_train_xgb

# 테스트 데이터에 대한 예측 수행
scikitlearn_xgb_predictions = scikitlearn_xgb_model.predict(X_test.values)

# 분류 정확도 계산
scikitlearn_xgb_accuracy = accuracy_score(y_test.values, scikitlearn_xgb_predictions)

In [18]:
# 분류 정확도 계산 및 시간
print('scikitlearn_xgb_accuracy:',scikitlearn_xgb_accuracy)
print('scikitlearn_xgb_training_time:',scikitlearn_xgb_training_time)

scikitlearn_xgb_accuracy: 0.9195459032576505
scikitlearn_xgb_training_time: 0.030332565307617188


## 3-2. LightGBM

### 3-2-a. 직접 구현하기

#### 특징:
1. Gradient-based One-Side Sampling (GOSS)과 Exclusive Feature Bundling (EFB)를 통합한 LightGBM의 간단한 구현입니다.
2. 여러 개의 결정 트리를 학습하여 앙상블 방식으로 분류 작업을 수행합니다.
3. 각 트리는 데이터의 일부 샘플과 특징을 선택하여 학습됩니다.

#### 주요 변수:
- `n_estimators`: 학습할 결정 트리의 수
- `learning_rate`: 학습률
- `max_depth`: 각 트리의 최대 깊이
- `min_samples_split`: 노드를 분할하기 위한 최소한의 샘플 수
- `subsample`: 데이터의 서브샘플링 비율 (GOSS를 위해 사용됨)
- `colsample`: 피처의 서브샘플링 비율 (EFB를 위해 사용됨)
- `alpha`: GOSS에서 큰 그라디언트를 가진 데이터의 비율
- `beta`: GOSS에서 작은 그라디언트를 가진 데이터의 비율
- `trees`: 학습된 결정 트리들을 저장하는 리스트
- `bundles`: EFB를 통해 묶인 특징들의 리스트

#### 모든 메서드:

1. `__init__(...)`: 초기화 함수로, 모델의 하이퍼파라미터와 필요한 변수들을 설정합니다.
2. `goss_subsampling(gradients)`: GOSS 알고리즘에 따라 서브샘플링된 데이터 인덱스를 반환합니다.
3. `exclusive_feature_bundling(X)`: EFB 알고리즘에 따라 피처를 번들로 묶은 후 번들링된 데이터를 반환합니다.
4. `fit(X, y)`: 주어진 학습 데이터 `X`와 레이블 `y`를 사용하여 모델을 학습시킵니다.
5. `predict(X)`: 학습된 모델을 사용하여 주어진 데이터 `X`의 레이블을 예측합니다.
6. `get_best_split(X, y, feature_subset)`: 주어진 데이터와 피처 집합을 기반으로 최적의 분할을 찾습니다.
7. `calculate_gain(y_left, y_right)`: 주어진 왼쪽 및 오른쪽 자식 노드의 목표값을 기반으로 정보 이득을 계산합니다.
8. `split_dataset(X, y, feature_index, threshold)`: 주어진 피처와 임계값을 기준으로 데이터셋을 두 부분으로 분할합니다.
9. `build_decision_tree(X, y, feature_subset, current_depth=0)`: 주어진 데이터와 피처 집합을 기반으로 결정 트리를 재귀적으로 구성합니다.
10. `predict_with_tree(tree, x)`: 주어진 트리와 입력 데이터 포인트를 사용하여 예측값을 계산합니다.

#### 동작 방식:
1. LightGBM의 구현은 여러 개의 결정 트리를 학습하여 앙상블 방식으로 분류 작업을 수행합니다.
2. GOSS는 데이터의 일부만을 사용하여 트리를 학습시키는 데, 큰 그라디언트를 가진 데이터와 일부 작은 그라디언트를 가진 데이터만을 선택합니다.
3. EFB는 피처를 번들로 묶어 데이터의 차원을 줄입니다.
4. 예측 시, 모든 트리의 예측 결과를 모아서 최종 예측을 결정합니다.

In [19]:
class LightGBM_Implementation:
    def __init__(self, n_estimators=5, learning_rate=0.1, max_depth=3, min_samples_split=5, subsample=0.8, colsample=0.8, alpha=0.2, beta=0.2):
        # 모델의 초기 설정값을 지정하는 생성자
        self.n_estimators = n_estimators  # 생성될 트리의 개수
        self.learning_rate = learning_rate  # 각 트리의 학습률
        self.max_depth = max_depth  # 트리의 최대 깊이
        self.min_samples_split = min_samples_split  # 노드를 분할하는 데 필요한 최소 샘플 수
        self.subsample = subsample  # GOSS에서 사용될 데이터 서브샘플링 비율
        self.colsample = colsample  # EFB에서 사용될 피처 서브샘플링 비율
        self.alpha = alpha  # GOSS에서 큰 그라디언트를 가진 데이터의 선택 비율
        self.beta = beta  # GOSS에서 작은 그라디언트를 가진 데이터의 선택 비율
        self.trees = []  # 학습된 트리들을 저장할 리스트
        self.bundles = []  # EFB를 통해 묶인 피처들의 리스트

    def goss_subsampling(self, gradients):
        """GOSS 알고리즘을 사용하여 데이터의 서브샘플을 선택하는 함수"""
        num_large = int(len(gradients) * self.alpha)  # 큰 그라디언트를 가진 데이터의 개수
        large_indices = np.argsort(-np.abs(gradients))[:num_large]  # 큰 그라디언트를 가진 데이터의 인덱스
        remaining_indices = np.delete(np.arange(len(gradients)), large_indices)  # 큰 그라디언트를 가진 데이터를 제외한 나머지 인덱스
        num_small = int(len(gradients) * self.beta)  # 작은 그라디언트를 가진 데이터의 개수
        small_indices = np.random.choice(remaining_indices, num_small, replace=False)  # 작은 그라디언트를 가진 데이터의 인덱스 선택
        return np.concatenate([large_indices, small_indices])  # 선택된 데이터의 인덱스 반환

    def exclusive_feature_bundling(self, X):
        """EFB 알고리즘을 사용하여 피처들을 번들로 묶는 함수"""
        n_features = X.shape[1]  # 전체 피처의 개수
        is_pair_exclusive = np.ones((n_features, n_features), dtype=bool)  # 피처 쌍이 배타적인지 확인하는 행렬

        # 모든 피처 쌍에 대해 배타적인지 확인
        for i, j in itertools.combinations(range(n_features), 2):
            if np.any(X[:, i] * X[:, j] != 0):
                is_pair_exclusive[i, j] = is_pair_exclusive[j, i] = False

        visited = set()  # 이미 번들에 추가된 피처의 인덱스를 저장할 집합
        bundles = []  # 피처 번들을 저장할 리스트

        # 모든 피처에 대해 번들 생성
        for i in range(n_features):
            if i in visited:
                continue
            bundle = [i]  # 새로운 번들 생성
            visited.add(i)
            for j in range(i+1, n_features):
                if is_pair_exclusive[i, j] and j not in visited:
                    bundle.append(j)  # 번들에 피처 추가
                    visited.add(j)
            bundles.append(bundle)

        # 번들링된 데이터 생성
        X_bundled = np.zeros((X.shape[0], len(bundles)))
        for idx, bundle in enumerate(bundles):
            X_bundled[:, idx] = np.sum(X[:, bundle], axis=1)

        return X_bundled, bundles  # 번들링된 데이터와 번들 정보 반환

    def fit(self, X, y):
        """모델을 학습하는 함수"""
        self.prediction = np.full(y.shape, np.mean(y))  # 초기 예측값 설정
        for _ in range(self.n_estimators):
            residuals = y - self.prediction  # 잔차 계산
            subsample_indices = self.goss_subsampling(residuals)  # GOSS를 사용하여 서브샘플 선택
            X_subset = X[subsample_indices]  # 선택된 서브샘플
            y_subset = y[subsample_indices]  # 선택된 서브샘플의 레이블
            X_bundle, self.bundles = self.exclusive_feature_bundling(X_subset)  # EFB를 사용하여 피처 번들링
            feature_subset = np.random.choice(X_bundle.shape[1], int(X_bundle.shape[1] * self.colsample), replace=False)  # 피처 서브샘플링
            
    def predict(self, X):
        """학습된 모델을 사용하여 주어진 입력 데이터 X에 대한 예측을 수행하는 함수."""
        final_predictions = np.full(X.shape[0], np.mean(self.prediction))  # 초기 예측값 설정
        for tree in self.trees:
            tree_predictions = np.array([self.predict_with_tree(tree, x) for x in X])  # 각 트리를 사용한 예측값 계산
            final_predictions += self.learning_rate * tree_predictions  # 예측값 업데이트
        return (final_predictions > 0.5).astype(int)  # 최종 예측값을 이진 분류 결과로 변환
    
    def get_best_split(self, X, y, feature_subset):
        """주어진 데이터와 피처 집합을 기반으로 최적의 분할을 찾는 함수."""
        best_gain = float('-inf')
        best_feature_index = None
        best_threshold = None
        for feature_index in feature_subset:
            unique_values = np.unique(X[:, feature_index])
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2.0
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self.split_dataset(X, y, feature_index, threshold)
                if len(y_left) < self.min_samples_split or len(y_right) < self.min_samples_split:
                    continue
                gain = self.calculate_gain(y_left, y_right)
                if gain > best_gain:
                    best_gain = gain
                    best_feature_index = feature_index
                    best_threshold = threshold
        return best_feature_index, best_threshold

    def calculate_gain(self, y_left, y_right):
        """주어진 왼쪽 및 오른쪽 자식 노드의 목표값을 기반으로 정보 이득을 계산하는 함수."""
        def squared_loss_grad(y):
            return np.mean(y)
        def hessian(y):
            return len(y)
        g_left, g_right = squared_loss_grad(y_left), squared_loss_grad(y_right)
        h_left, h_right = hessian(y_left), hessian(y_right)
        gain = 0.5 * (g_left**2 / (h_left + 1e-3) + g_right**2 / (h_right + 1e-3) - (g_left + g_right)**2 / (h_left + h_right + 1e-3))
        return gain

    def split_dataset(self, X, y, feature_index, threshold):
        """주어진 피처와 임계값을 기준으로 데이터셋을 두 부분으로 분할하는 함수."""
        left_mask = X[:, feature_index] < threshold
        right_mask = ~left_mask
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    def build_decision_tree(self, X, y, feature_subset, current_depth=0):
        """주어진 데이터와 피처 집합을 기반으로 결정 트리를 재귀적으로 구성하는 함수."""
        if current_depth >= self.max_depth or len(y) <= self.min_samples_split:
            return {'prediction': np.mean(y)}
        feature_index, threshold = self.get_best_split(X, y, feature_subset)
        if feature_index is None:
            return {'prediction': np.mean(y)}
        X_left, y_left, X_right, y_right = self.split_dataset(X, y, feature_index, threshold)
        return {
            'feature_index': feature_index,
            'threshold': threshold,
            'left': self.build_decision_tree(X_left, y_left, feature_subset, current_depth + 1),
            'right': self.build_decision_tree(X_right, y_right, feature_subset, current_depth + 1)
        }

    def predict_with_tree(self, tree, x):
        """주어진 트리와 입력 데이터 포인트를 사용하여 예측값을 계산하는 함수.
        이 함수는 재귀적으로 호출되어 주어진 입력 데이터 포인트를 트리의 잎 노드까지 전달하고,
        해당 노드에서의 예측값을 반환한다."""
        if 'prediction' in tree:
            return tree['prediction']
        feature_index = tree['feature_index']
        threshold = tree['threshold']
        if x[feature_index] < threshold:
            return self.predict_with_tree(tree['left'], x)
        else:
            return self.predict_with_tree(tree['right'], x)

In [20]:
# 학습
lgbm_implemantation = LightGBM_Implementation(n_estimators=5, learning_rate=0.1, max_depth=3, min_samples_split=5, subsample=0.8, colsample=0.8)

start_time_train = time.time()
lgbm_implemantation.fit(X_train.values, y_train.values)
end_time_train = time.time()
lgbm_implemantation_training_time = end_time_train - start_time_train

# 테스트 데이터에 대한 예측 수행
lgbm_implemantation_predictions = lgbm_implemantation.predict(X_test.values)

# 정확도 계산
lgbm_implemantation_accuracy = accuracy_score(y_test.values, lgbm_implemantation_predictions)

In [21]:
# 분류 정확도 계산 및 시간
print('lgbm_implemantation_accuracy:',lgbm_implemantation_accuracy)
print('lgbm_implemantation_training_time:',lgbm_implemantation_training_time)

lgbm_implemantation_accuracy: 0.8395853899308984
lgbm_implemantation_training_time: 0.016425609588623047


### 3-2-b. 라이브러리 활용

In [22]:
# 학습 및 예측
scikitlearn_lgbm_model = LGBMClassifier(n_estimators=5, learning_rate=0.1, max_depth=3)

start_time_train_lgbm = time.time()  # 학습 시작 시간 저장
scikitlearn_lgbm_model.fit(X_train.values, y_train.values)  # 모델 학습
end_time_train_lgbm = time.time()  # 학습 종료 시간 저장
scikitlearn_lgbm_training_time = end_time_train_lgbm - start_time_train_lgbm

# 테스트 데이터에 대한 예측 수행
scikitlearn_lgbm_predictions = scikitlearn_lgbm_model.predict(X_test.values)

# 분류 정확도 계산
scikitlearn_lgbm_accuracy = accuracy_score(y_test.values, scikitlearn_lgbm_predictions)

In [23]:
# 분류 정확도 계산 및 시간
print('scikitlearn_lgbm_accuracy:',scikitlearn_lgbm_accuracy)
print('scikitlearn_lgbm_training_time:',scikitlearn_lgbm_training_time)

scikitlearn_lgbm_accuracy: 0.851431391905232
scikitlearn_lgbm_training_time: 0.018869400024414062


## 3-3. CatBoost

### 3-3-b. 라이브러리 활용

In [24]:
# 학습
scikitlearn_catboost_model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=3, verbose=0)

start_time_train_catboost = time.time()  # 학습 시작 시간 저장
scikitlearn_catboost_model.fit(X_train.values, y_train.values)  # 모델 학습
end_time_train_catboost = time.time()  # 학습 종료 시간 저장
scikitlearn_catboost_training_time = end_time_train_catboost - start_time_train_catboost

# 테스트 데이터에 대한 예측 수행
scikitlearn_catboost_predictions = scikitlearn_catboost_model.predict(X_test.values)

# 분류 정확도 계산
scikitlearn_catboost_accuracy = accuracy_score(y_test.values, scikitlearn_catboost_predictions)

In [25]:
# 분류 정확도 계산 및 시간
print('scikitlearn_catboost_accuracy:',scikitlearn_catboost_accuracy)
print('scikitlearn_catboost_training_time:',scikitlearn_catboost_training_time)

scikitlearn_catboost_accuracy: 0.9639684106614018
scikitlearn_catboost_training_time: 0.40671229362487793
